
Creating a Connect Four game with AB pruning and A* as player strategies. 


---

Some useful packages and libraries:



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
import heapq
import unittest
from scipy import stats
import copy as cp
from time import time
from enum import Enum
from typing import Dict, Tuple
import sys
from typing import Callable
import random
from collections import namedtuple
from typing import Optional

---

## Problem 1: Game Theory - Playing "intelligent" Connect Four (100 points total)

Connect Four is a two-player game where the objective is to get four pieces in a row - horizontally, vertically, or diagonally. Check out this video if you're unfamiliar with the game: https://www.youtube.com/watch?v=utXzIFEVPjA.

* `moves` is a list of columns to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`. Note that when a 'move' is made, the column must have an open slot and the piece must drop to the lowest row. 
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` returns the utility of the current state if the player is Red and $-1 \cdot$ utility if the player is Black.
* `display(self)` is a method to display the current game `state`. You get it for free because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if Red wins, 0 if draw, -1 if Black wins). `player1` and `player2` are functional arguments that we will deal with in parts **1b** and **1d**.

Some notes:
* Assume Red always goes first.
* Do **not** hard-code for 6x7 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [2]:

class Player(Enum):
    RED = "R"
    BLACK = "B"
    EMPTY = "."

class State:
    def __init__(self, board_dimensions: tuple[int, int], utility: int = 0, board: dict[tuple[int,int], Player] = {}):
        self.to_move = Player.RED
        self.utility = utility
        self.dimensions = board_dimensions
        self.board : dict[tuple[int,int], Player] = board
        self._moves = None
        self._adjacent = None

    def moves(self) -> list[tuple[int, int]]:
        if self._moves is not None:
            return self._moves

        movesPoss = {col: self.dimensions[0] for col in range(1, self.dimensions[1] + 1)}

        for row, col in self.board:
            if col in movesPoss:
                next_row = row - 1
                if next_row <= 0:
                    movesPoss.pop(col)
                elif next_row < movesPoss[col]:
                    movesPoss[col] = next_row

        self._moves = [(movesPoss[col], col) for col in movesPoss]
        return self._moves


    def get_adjacent(self, move: tuple[int,int], n_adjacent: int, search_for: list[Player] = [*Player]) -> list[tuple[list[Player], list[Player]]]:
        if Player.EMPTY in search_for and not None in search_for:
            search_for.append(None)
        directions = [
            (iter_row, iter_col)
            for iter_row in range(-1,2)
            for iter_col in range(-1,2)
            if (iter_row, iter_col) != (0,0)
        ]
        adjacent: list[tuple[list[Player], list[Player]]] = []
        for direction in directions:
            forward, backward = self.check_direction(move, direction, n_adjacent, search_for)
            adjacent.append((forward, backward))
        return adjacent

    def check_direction(self, move: tuple[int,int], direction: tuple[int,int], n_adjacent: int, search_for: list[Player]) -> tuple[list[Player], list[Player]]:
        forward, backward = [], []
        for offset in range(1, n_adjacent):
            check_state = (move[0] + direction[0] * offset, move[1] + direction[1] * offset)
            if not (1 <= check_state[0] <= self.dimensions[0] and 1 <= check_state[1] <= self.dimensions[1]):
                break
            check_state_value = self.board.get(check_state)
            if check_state_value in search_for:
                value = Player.EMPTY if check_state_value is None else check_state_value
                forward.append(value)
            else:
                break
        for offset in range(1, n_adjacent):
            check_state = (move[0] - direction[0] * offset, move[1] - direction[1] * offset)
            if not (1 <= check_state[0] <= self.dimensions[0] and 1 <= check_state[1] <= self.dimensions[1]):
                break
            check_state_value = self.board.get(check_state)
            if check_state_value in search_for:
                value = Player.EMPTY if check_state_value is None else check_state_value
                backward.insert(0, value)
            else:
                break
        return forward, backward
    
    def hash(self):
        return "~".join([str(e)+ ":" + str(self.board[e]) for e in self.board])

class ConnectFour:
    def __init__(self, nrow=6, ncol=7, nwin=4):
        self.nrow: int = nrow
        self.ncol: int = ncol
        self.nwin: int = nwin
        self.state = State((nrow, ncol))

    def result(self, move: tuple[int, int], state: State) -> State:
        '''
        move  = (row, col) tuple where player will put their mark (R or B)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''

        # Check if the move is valid
        if move not in state.moves():
            return state

        # Compute the utility of the move
        utility = self.compute_utility(move, state)

        # Create a new state with the updated board and next player
        new_board = state.board.copy()
        new_board[move] = state.to_move
        new_state = State(state.dimensions, utility, new_board)
        new_state.to_move = Player.BLACK if state.to_move == Player.RED else Player.RED

        return new_state

    def compute_utility(self, move: tuple[int,int], state: State) -> int:
        '''
        If 'R' wins with this move, return 1;
        if 'B' wins return -1;
        else return 0.
        '''        

        # Check if the move causes a win
        adjacent = state.get_adjacent(move, self.nwin, [state.to_move])
        for angle in adjacent:
            sequential = 1
            for direction in angle:
                for piece in direction:
                    if piece == state.to_move:
                        sequential += 1
                        if sequential == self.nwin:
                            return 1 if state.to_move == Player.RED else -1
                    else:
                        sequential = 0

        # Otherwise, return 0
        return 0

    def game_over(self, state: State) -> bool:
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''

        # your code goes here... 
        return state.utility != 0 or len(state.moves()) == 0

    def utility(self, state: State, player: Player) -> int:
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''
        # your code goes here...
        if state.utility == float('inf'):
            return 1 if player == Player.RED else -1
        elif state.utility == float('-inf'):
            return -1 if player == Player.RED else 1
        else:
            if state.utility > 0:
                return 1 if player == Player.RED else -1
            elif state.utility < 0:
                return -1 if player == Player.RED else 1
            else:
                return 0
    
    def display(self, state: State = None) -> None:
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()

    def play_game(self, player1, player2,) -> int:
        players: dict[Player, Callable[[ConnectFour], tuple[int,int]]] = { Player.RED: player1, Player.BLACK: player2}
        current_player = Player.RED

        while not self.game_over(self.state):
            playing = players[current_player]
            move: tuple[int,int] = playing(self)
            new_state = self.result(move, self.state)
            self.state = new_state
            current_player = Player.RED if current_player == Player.BLACK else Player.BLACK

        return self.utility(self.state, Player.RED)

The function `random_player` that takes a single argument of the `ConnectFour` class and returns a random move out of the available legal moves in the `state` of the `ConnectFour` game.

In [3]:
def random_player(game: ConnectFour) -> tuple[int,int]:
    '''A player that chooses a legal move at random out of all
    available legal moves in ConnectFour state argument'''
    
    available_moves = game.state.moves()
    selected_move = random.choice(available_moves)
    return selected_move

Checking to see the win percentage of the person that goes first in the game and due to this testing being done before it should be around 55% when put in a simulation of 1000 games.

In [4]:
# 1000 games between two random players
def play_games(player1, player2, iterations: int, game_args = (6, 7, 4), get_games=False):
        # Initialize the win counters
    wins = {Player.RED: 0, Player.BLACK: 0, Player.EMPTY: 0}

    # Play the games
    games = []
    i = 0
    while i < iterations:
        game = ConnectFour(*game_args)
        games.append(game)
        result = game.play_game(player1, player2)
        winner = None
        if result == 1:
            winner = Player.RED
        elif result == -1:
            winner = Player.BLACK
        else:
            winner = Player.EMPTY
        wins[winner] += 1
        i += 1
    # Print the results
    total_games = iterations
    tie = total_games - (wins[Player.RED] + wins[Player.BLACK])
    print(f"Total number of games played: {total_games}")
    print(f"Player {Player.RED.name} won {wins[Player.RED]} games ({100 * wins[Player.RED] / total_games:.2f}%)")
    print(f"Player {Player.BLACK.name} won {wins[Player.BLACK]} games ({100 * wins[Player.BLACK] / total_games:.2f}%)")
    print(f"Out of the games there were {tie} ties ({100 * tie / total_games:.2f}%)")

    if get_games:
        return games


play_games(random_player, random_player, 1000)

Total number of games played: 1000
Player RED won 555 games (55.50%)
Player BLACK won 444 games (44.40%)
Out of the games there were 1 ties (0.10%)


Looking to see what the long-term win percentage appears to be for the first player in a 10x10 ConnectFour tournament, where 4 marks must be connected for a win?  

In [5]:
# 1000 games between two random players
play_games(random_player, random_player, 1000, (10,10,6))

Total number of games played: 1000
Player RED won 473 games (47.30%)
Player BLACK won 435 games (43.50%)
Out of the games there were 92 ties (9.20%)


We observe that there is way more ties when the size of the state space changes from it increasing from 4 to 101 ties which in turn lowers the win percentage of the player. Where in this case he lost more going first than winning this most likely is because there are so many more spaces that it basically takes away the advantage that the person going first had.

Created an `alphabeta_player` that takes a single argument of a `ConnectFour` class object and returns the minimax move in the `state` of the `ConnectFour` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

In [6]:
discount_factor = 0.9
nodes_expanded = 0
UtilityMove = namedtuple("UtilityMove", ["utility", "move"])

def alphabeta_player(game: ConnectFour, report_nodes_expanded=False) -> tuple[int, int]:
    global nodes_expanded
    nodes_expanded = 0
    utility = alphabeta_value(game, game.state, float("-inf"), float("inf"))
    if report_nodes_expanded:
        return (utility.move, nodes_expanded)
    return utility.move

def report_nodes_expanded_func() -> int:
    global nodes_expanded
    return nodes_expanded

def alphabeta_value(game: ConnectFour, state: State, alpha: int, beta: int) -> UtilityMove:
    global nodes_expanded
    nodes_expanded += 1

    if game.game_over(state):
        return UtilityMove(state.utility, None)

    if state.to_move == Player.RED:
        return max_value(game, state, alpha, beta)
    else:
        return min_value(game, state, alpha, beta)

def max_value(game, state, alpha, beta):
    utility = UtilityMove(-np.inf, None)
    child_rechable_boundary = [alpha]

    for move in state.moves():
        child_utility, _ = alphabeta_value(game, game.result(move, state), alpha, beta)
        child_utility *= discount_factor
        utility = max(utility, UtilityMove(child_utility, move))

        if utility.utility >= beta:
            return utility
        child_rechable_boundary[0] = max(utility.utility, child_rechable_boundary[0])

        alpha = max(alpha, utility.utility)

    return utility

def min_value(game, state, alpha, beta):
    utility = UtilityMove(np.inf, None)
    child_rechable_boundary = [beta]

    for move in state.moves():
        child_utility, _ = alphabeta_value(game, game.result(move, state), alpha, beta)
        child_utility *= discount_factor
        utility = min(utility, UtilityMove(child_utility, move))

        if utility.utility <= alpha:
            return utility
        child_rechable_boundary[0] = min(utility.utility, child_rechable_boundary[0])

        beta = min(beta, utility.utility)

    return utility

def minimax(game, state, maximizing_player=True):
    global nodes_expanded_minimax  # add this line to access the global variable
    nodes_expanded_minimax += 1
    
    if game.game_over(state):
        return state.utility, None

    best_move = None
    if maximizing_player:
        max_utility = float('-inf')
        for move in state.moves():
            child_state = game.result(move, state)
            child_utility, _ = minimax(game, child_state, False)
            if child_utility > max_utility:
                max_utility = child_utility
                best_move = move
        return max_utility, best_move
    else:
        min_utility = float('inf')
        for move in state.moves():
            child_state = game.result(move, state)
            child_utility, _ = minimax(game, child_state, True)
            if child_utility < min_utility:
                min_utility = child_utility
                best_move = move
        return min_utility, best_move


1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. Two alpha-beta players should always draw. One player is the max player and the other player is the min player.

These should happen when the code runs:

In [7]:
print(" AlphaBeta vs Random")
play_games(alphabeta_player, random_player, 10, (3,4,3))

print(" Random vs AlphaBeta")
play_games(random_player, alphabeta_player, 10, (3,4,3))

 AlphaBeta vs Random
Total number of games played: 10
Player RED won 10 games (100.00%)
Player BLACK won 0 games (0.00%)
Out of the games there were 0 ties (0.00%)
 Random vs AlphaBeta
Total number of games played: 10
Player RED won 3 games (30.00%)
Player BLACK won 6 games (60.00%)
Out of the games there were 1 ties (10.00%)


Calculated the number of number of states expanded by the minimax algorithm, **with and without pruning**, to determine the minimax decision from the initial 6x7 ConnectFour board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

I then computed the percent savings that you get by using alpha-beta pruning. i.e. Compute $\frac{\text{Number of nodes expanded with pruning}}{\text{Number of nodes expanded with minimax}} $

In [8]:
game = ConnectFour(3,4,3)

initial_state = game.state

# Calculate nodes expanded with minimax
nodes_expanded_minimax = 0
minimax(game, initial_state, True)
print(f"Number of nodes expanded with minimax: {nodes_expanded_minimax}")

# Calculate nodes expanded with alpha-beta pruning
nodes_expanded_ab = alphabeta_player(game, report_nodes_expanded=True)[1]
print(f"Number of nodes expanded with alpha-beta pruning: {nodes_expanded_ab}")

# Compute percentage savings
percent_savings = (1 - (nodes_expanded_ab / nodes_expanded_minimax)) * 100
print(f"Percentage savings with alpha-beta pruning: {percent_savings}%")

Number of nodes expanded with minimax: 277167
Number of nodes expanded with alpha-beta pruning: 12277
Percentage savings with alpha-beta pruning: 95.5705405044612%


We see the number of nodes expanded by alpha-beta pruning is much smaller than the number of nodes expanded by minimax algorithm in ConnectFour game. This is because alpha-beta pruning avoids exploring branches of the game tree that are unlikely to lead to the best move, while minimax algorithm explores all possible branches.

**Creating A* search:**

I created a heuristic that will count the number of open runs of three or more consecutive pieces for the current player and the opponent player, and then calculates the minimum distance to a win for the current player and the opponent player in each direction (row, column, diagonal), and returns the minimum value of the distances as the heuristic value.

The intuition behind this heuristic is that having more open runs of three or more consecutive pieces increases the player's chances of winning, while having more open runs of the opponent decreases their chances of winning. By calculating the minimum distance to a win for both players in each direction, the heuristic takes into account the potential for the opponent to block the player's runs and for the player to block the opponent's runs.

In [9]:
def heuristic(game: ConnectFour, move: tuple[int,int], state: State) -> float:
    other_p = Player.BLACK if state.to_move == Player.RED else Player.RED
    self_p = state.to_move

    heuristics = []
    blocks = [[*front, other_p, *back] for front, back in state.get_adjacent(move, game.nwin, [Player.EMPTY, other_p])]
    wins = [[*front, self_p, *back] for front, back in state.get_adjacent(move, game.nwin, [Player.EMPTY, self_p])]
    for direction_idx in range(len(wins)):
        row_wins = wins[direction_idx]
        row_blocks = blocks[direction_idx]

        dist_win = game.nwin - row_wins.count(self_p) if len(row_wins) >= game.nwin else np.inf
        dist_block = game.nwin - row_blocks.count(other_p) if len(row_blocks) >= game.nwin else np.inf
        heuristics.append(min(dist_win, dist_block))

    heuristic = min(heuristics)
    return heuristic

In [10]:
# Doing a Uniform Search
class Frontier_PQ:
    def __init__(self, start_state: State, start_cost: int = 0) -> None:
        self.start_state: State = start_state
        self.states: Dict[str, Tuple[int, State, Optional[State]]] = {start_state.hash(): (start_cost, start_state, None)}
        self.queue: List[Tuple[int, str, Optional[str]]] = [(start_cost, start_state.hash(), None)]

    def add(self, state: State, cost: int, parent_state: State):
        heapq.heappush(self.queue, [cost, state.hash(), parent_state.hash()])
        self.states[state.hash()] = (cost, state, parent_state)
    
    def pop(self) -> Tuple[int, State, Optional[State]]:
        pop_tuple = None
        while pop_tuple is None or pop_tuple[0] == "REMOVED":
            pop_tuple = heapq.heappop(self.queue)
            if pop_tuple[0] == "REMOVED":
                print("FOUND REMOVED")
                pop_tuple = None
        return self.states.pop(pop_tuple[1])

    def replace(self, state: State, new_cost: int, new_parent_state: State):
        previous_cost, old_state, old_parent_state = self.states.get(state.hash(), (float('inf'), None, None))    

        if new_cost < previous_cost:
            self.states[state.hash()] = (new_cost, state, new_parent_state)
            for idx, (old_cost, node, old_parent) in enumerate(self.queue):
                if node == state.hash():
                    self.queue[idx] = (new_cost, node, new_parent_state.hash())
                    heapq.heapify(self.queue)
                    break
            else:
                heapq.heappush(self.queue, (new_cost, state.hash(), new_parent_state.hash()))
        
def path(previous: dict[State, State], s): 
    if s is None:
        return []
    else:
        return path(previous, previous[s])+[s]


def a_star(game: ConnectFour, start_state: State, return_cost=False):
    frontier = Frontier_PQ(start_state)
    parent_tree: dict[State, State] = {start_state: None}
    final_state: State = start_state

    while len(frontier.states) != 0:
        previous_cost, current_state, parent_state = frontier.pop()
        parent_tree[current_state] = parent_state

        if game.game_over(current_state):
            final_state = current_state
            if game.utility(current_state, start_state.to_move) > 0:
                break

            no_chance_parents = find_no_chance_parents(game, start_state, parent_tree, final_state)
            for st in frontier.states:
                state_cost, state, parent = frontier.states[st]
                if parent in no_chance_parents:
                    frontier.replace(state, np.inf, parent)
            continue
        
        explore_children(game, current_state, frontier, parent_tree)
    
    solution_path = path(parent_tree, final_state)
    solution_cost = len(solution_path)
    return (solution_path, solution_cost, len(parent_tree.keys())) if return_cost else solution_path


def find_no_chance_parents(game, start_state, parent_tree, final_state):
    no_chance_parents = [parent_tree[final_state]]
    # Stop exploring any branches where they could win in one move from ours.
    assert no_chance_parents[0].to_move != start_state.to_move, no_chance_parents[0].to_move + " != " + start_state.to_move.name
    def bad_children(bad_node: State, d=0):
        for key, value in parent_tree.items():
            if value == bad_node:
                no_chance_parents.append(key)
                bad_children(key, d + 1)
    bad_children(no_chance_parents[0])
    return no_chance_parents


def explore_children(game, current_state, frontier, parent_tree):
    for action in current_state.moves():
        child_state = game.result(action, current_state)
        child_path_cost = len(child_state.board) + heuristic(game, action, child_state)

        if not child_state in list(parent_tree.keys()) and not child_state in frontier.states:
            frontier.add(child_state, child_path_cost, current_state)
        elif child_state in frontier.states and frontier.states[child_state] > child_path_cost:
            frontier.replace(child_state, child_path_cost, current_state)

def a_star_player(game: ConnectFour, return_cost=False):
    if return_cost:
        solution_path, solution_cost, explored = a_star(game, game.state, True)
    else:
        solution_path = a_star(game, game.state, False)

    for action in game.state.moves():
        new_state = game.result(action, game.state)
        if new_state.board == solution_path[1].board:
            return action

    return None

In [11]:
game = ConnectFour(3,4,3)
game.play_game(a_star_player, a_star_player)

1

### Comparing A* to other algorithms (10 points)
Next, I play 10 games of Connect Four using your A* player and a random player and 10 games against the AB pruning player.

In [12]:
def count_moves(games: list[ConnectFour]) -> dict:
    all_moves = {p:0 for p in Player}

    for game in games:
        game_board = game.state.board
        for p in game_board.values():
            all_moves[p] += 1

    return all_moves

def display_avg_moves(games: list[ConnectFour]):
    all_moves = count_moves(games)
    num_games = len(games)

    for p, moves in all_moves.items():
        print("Player {} made {} moves on average across {} games".format(p.name, moves / num_games, num_games))

In [13]:
print("10 games played between A* player and Alpha-Beta player on a 3x4x3 board with 3 in a row to win.")
moves = play_games(a_star_player, alphabeta_player, 10, (3,4,3), True)
display_avg_moves(moves)

10 games played between A* player and Alpha-Beta player on a 3x4x3 board with 3 in a row to win.
Total number of games played: 10
Player RED won 10 games (100.00%)
Player BLACK won 0 games (0.00%)
Out of the games there were 0 ties (0.00%)
Player RED made 6.0 moves on average across 10 games
Player BLACK made 5.0 moves on average across 10 games
Player EMPTY made 0.0 moves on average across 10 games


In [14]:
print("10 games played between Alpha-Beta player and Random player on a 3x4x3 board with 3 in a row to win.")
moves = play_games(alphabeta_player, random_player, 10, (3,4,3), True)
display_avg_moves(moves)

10 games played between Alpha-Beta player and Random player on a 3x4x3 board with 3 in a row to win.
Total number of games played: 10
Player RED won 10 games (100.00%)
Player BLACK won 0 games (0.00%)
Out of the games there were 0 ties (0.00%)
Player RED made 3.1 moves on average across 10 games
Player BLACK made 2.1 moves on average across 10 games
Player EMPTY made 0.0 moves on average across 10 games


I notice in the first matchup between Astar and Alphabeta players on a 3x4 board with 3 to win, Astar player emerged as the clear winner. Across 10 games, Astar won all the games while Alphabeta was unable to win any. None of the games resulted in a tie. On average, Astar player made 6 moves while Alphabeta made 5 moves. It is interesting to note that the number of moves made by Astar is more than Alphabeta, indicating that Astar may be exploring more states before making a move.

In the second matchup between Alphabeta and Random players on the same board, Alphabeta player won all the games while Random was unable to win any. None of the games resulted in a tie. On average, Alphabeta player made 3.1 moves while Random made 2.1 moves. The difference in the number of moves made by Alphabeta and Random players is quite significant, indicating that Alphabeta is playing more optimally.

Based on the outcomes, it appears that Astar and Alphabeta players are stronger than their counterparts. However, the results do not provide conclusive evidence as only a small number of games were played. It is also important to note that the performance of the players may vary depending on the size of the board and the number of moves required to win.

In situations where one player appeared to do better than the other, it is possible that the heuristics used by one player were more effective than the other. For example, in the first matchup, Astar may have been better at evaluating the game state and exploring more states before making a move, giving it an advantage over Alphabeta. In the second matchup, Alphabeta may have been better at analyzing the game state and choosing the best move, giving it an advantage over Random.

To further improve the performance of the players, other heuristics can be implemented. For example, one can use a combination of heuristics to evaluate the game state, such as considering the number of pieces in a row, column, or diagonal, and the number of open spaces. Additionally, one can implement a search algorithm that is more efficient than Astar or Alphabeta, such as Iterative Deepening Search. Overall, the results suggest that both Astar and Alphabeta players are strong, but their performance can be further improved with additional heuristics and search algorithms.